In [1]:
import pygame, sys
import math
import random

pygame 2.1.2 (SDL 2.0.18, Python 3.6.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Object:
    def __init__(self, pos, size=(25,25), speed=0, colour=(0,0,0)):
        self.pos = pos
        self.rect = pygame.Rect(self.pos[0],self.pos[1],size[0],size[1])
        self.speed = speed
        self.colour = colour
        
    def render(self, screen):
        pygame.draw.rect(screen, self.colour, self.rect)

In [3]:
class Player(Object):
    def __init__(self, pos):
        super().__init__(pos, size=(25,25), speed=5, colour=(255,0,0))
        self.movement = ([False, False], [False, False])
        
    def update(self, walls):
        if self.movement[0][1] - self.movement[0][0] > 0 and self.movement[1][1] - self.movement[1][0] > 0:
            speed = round(self.speed/math.sqrt(2))
        else:
            speed = self.speed

        self.pos[0] += (self.movement[0][1] - self.movement[0][0])* speed
        self.rect.x = self.pos[0]
        for item in walls.walls:
            if self.rect.colliderect(item.rect):
                if self.movement[0][1] - self.movement[0][0] < 0:
                    self.rect.left = item.rect.right
                if self.movement[0][1] - self.movement[0][0] > 0:
                    self.rect.right = item.rect.left
                self.pos[0] = self.rect.x  
                
        self.pos[1] += (self.movement[1][1] - self.movement[1][0])* speed
        self.rect.y = self.pos[1]
        for item in walls.walls:
            if self.rect.colliderect(item.rect):
                if self.movement[1][1] - self.movement[1][0] < 0:
                    self.rect.top = item.rect.bottom
                elif self.movement[1][1] - self.movement[1][0] > 0:
                    self.rect.bottom = item.rect.top
                self.pos[1] = self.rect.y 

In [4]:
class Cop(Object):
    def __init__(self, pos):
        super().__init__(pos, size=(25,25), speed=2, colour=(0,0,255))

    def update(self, walls, following=None):
        # following is the Object class being followed
        if following != None:
            if self.pos[0] != following.pos[0] and self.pos[1] != following.pos[1]:
                speed = round(self.speed/math.sqrt(2))
            else:
                speed = self.speed
                
            if self.pos[0] != following.pos[0]:
                self.pos[0] += 2*((self.pos[0] < following.pos[0])-0.5) * self.speed
                self.rect.x = self.pos[0]
                
            for item in walls.walls:
                if self.rect.colliderect(item.rect):
                    if self.pos[0] > following.pos[0]:
                        self.rect.left = item.rect.right
                    if self.pos[0] < following.pos[0]:
                        self.rect.right = item.rect.left
            self.pos[0] = self.rect.x
            
            if self.pos[1] != following.pos[1]:
                self.pos[1] += 2*((self.pos[1] < following.pos[1])-0.5) * self.speed
                self.rect.y = self.pos[1]

            for item in walls.walls:
                if self.rect.colliderect(item.rect):
                    if self.pos[1] > following.pos[1]:
                        self.rect.top = item.rect.bottom
                    if self.pos[1] < following.pos[1]:
                        self.rect.bottom = item.rect.top       
            self.pos[1] = self.rect.y
                    
            if self.rect.colliderect(following.rect):
                self.colour = (0,255,0)
            else:
                self.colour = (0,0,255)

    def render(self, screen):
        pygame.draw.rect(screen, self.colour, self.rect)

In [5]:
class Coin(Object):
    def __init__(self, pos):
        super().__init__(pos, size=(20,20), colour=(255,255,0))
        self.active = True
        
    def update(self, player, screen):
        if not self.active:
            # do not render but do not add points

            # set new random location and set active again
            self.pos[0] = random.randint(0,400)
            self.pos[1] = random.randint(0,400)
            self.rect.x = self.pos[0]
            self.rect.y = self.pos[1]
            
            self.active = True
            
            return False
        elif self.active and player.rect.colliderect(self.rect): # picked up
            # delete self - do not render
            self.active = False
            return True  
        else:
            super().render(screen)
            return False

In [6]:
class Walls:
    def __init__(self):
        self.walls = []

    def create_wall(self, pos, wall_size):
        self.walls.append(Object(pos, size=wall_size))

    def render(self, screen):
        for wall in self.walls:
            wall.render(screen)

In [7]:
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("top down test game")
        self.screen = pygame.display.set_mode((640,480))
        self.clock = pygame.time.Clock()

        self.player = Player([160,260])
        self.cop = Cop([0,0])
        self.coin = Coin([300,300])
        self.walls = Walls()

        self.points = 0
        font = pygame.font.get_default_font()
        self.font = pygame.font.SysFont(font, 30)
        
    def run(self):
        while True:
            self.screen.fill((255,255,255))

            self.player.update(self.walls)
            self.player.render(self.screen)

            self.cop.update(self.walls, following=self.player)
            self.cop.render(self.screen)

            picked_coin = self.coin.update(self.player, self.screen)

            self.walls.render(self.screen)
            
            if picked_coin:
                self.points += 1
            
            text_surf = self.font.render(f'Points: {self.points}', True, (0,0,0))            
            self.screen.blit(text_surf, (0,0))
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_LEFT:
                        self.player.movement[0][0] = True
                    if event.key == pygame.K_RIGHT:
                        self.player.movement[0][1] = True
                    if event.key == pygame.K_UP:
                        self.player.movement[1][0] = True
                    if event.key == pygame.K_DOWN:
                        self.player.movement[1][1] = True
                if event.type == pygame.KEYUP:
                    if event.key == pygame.K_LEFT:
                        self.player.movement[0][0] = False
                    if event.key == pygame.K_RIGHT:
                        self.player.movement[0][1] = False
                    if event.key == pygame.K_UP:
                        self.player.movement[1][0] = False
                    if event.key == pygame.K_DOWN:
                        self.player.movement[1][1] = False
                if event.type == pygame.MOUSEBUTTONDOWN:
                    corner1 = pygame.mouse.get_pos()
                if event.type == pygame.MOUSEBUTTONUP:
                    corner2 = pygame.mouse.get_pos()
                    size = (abs(corner1[0]-corner2[0]), abs(corner1[1]-corner2[1]))

                    self.walls.create_wall(corner1, size)

            pygame.display.update()
            self.clock.tick(60)

In [8]:
Game().run()

SystemExit: 

c:\program files\python36\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
